# Car Price Prediction

## Import Libraries

#### 1.Import necessary libraries

In [133]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
import pickle

import warnings
warnings.filterwarnings('ignore')

## Load and Explore Data

#### 2.Load the dataset:

In [46]:
df = pd.read_csv('quikr_car.csv')

#### 3.Initial exploration:

In [47]:
df.head()

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing XO eRLX Euro III,Hyundai,2007,"80,000","45,000 kms",Petrol
1,Mahindra Jeep CL550 MDI,Mahindra,2006,"4,25,000",40 kms,Diesel
2,Maruti Suzuki Alto 800 Vxi,Maruti,2018,Ask For Price,"22,000 kms",Petrol
3,Hyundai Grand i10 Magna 1.2 Kappa VTVT,Hyundai,2014,"3,25,000","28,000 kms",Petrol
4,Ford EcoSport Titanium 1.5L TDCi,Ford,2014,"5,75,000","36,000 kms",Diesel


In [48]:
df.shape

(892, 6)

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        892 non-null    object
 1   company     892 non-null    object
 2   year        892 non-null    object
 3   Price       892 non-null    object
 4   kms_driven  840 non-null    object
 5   fuel_type   837 non-null    object
dtypes: object(6)
memory usage: 41.9+ KB


In [50]:
df.isnull().sum()

name           0
company        0
year           0
Price          0
kms_driven    52
fuel_type     55
dtype: int64

## Data Cleaning
### Identify Issues
#### 4.Quality issues identified:
- 'year' column has non-year values.
- Convert 'year' from object to int.
- 'Price' column has non-numeric values ('Ask For Price').
- Convert 'Price' from object to int.
- 'kms_driven' column has commas and 'kms' unit.
- Convert 'kms_driven' from object to int.
- 'fuel_type' column has NaN values.
- 'name' column should keep the first 3 words only.

## Clean the Data

#### 5. Backup the original data

In [51]:
backup = df.copy()

#### 6. Clean the 'year' column

In [52]:
df = df[df['year'].str.isnumeric()]

In [53]:
df['year']=df['year'].astype(int)

#### 7. Clean the 'Price' column

In [54]:
df = df[df['Price'] != 'Ask For Price']

In [55]:
df['Price'] = df['Price'].str.replace(',','').astype(int)

#### 8. Clean the 'year' column

In [62]:
df['kms_driven'] =df['kms_driven'].str.split(' ').str.get(0).str.replace(',','') 

In [66]:
df = df[df['kms_driven'].str.isnumeric()]

In [68]:
df['kms_driven'] = df['kms_driven'].astype(int)

#### 9. Handle missing values in 'fuel_type'

In [72]:
df = df[~df['fuel_type'].isna()]

In [73]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 816 entries, 0 to 889
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        816 non-null    object
 1   company     816 non-null    object
 2   year        816 non-null    int32 
 3   Price       816 non-null    int32 
 4   kms_driven  816 non-null    int32 
 5   fuel_type   816 non-null    object
dtypes: int32(3), object(3)
memory usage: 35.1+ KB


#### 10. Clean the 'name' column

In [78]:
df['name'] = df['name'].str.split(' ').str.slice(0,3).str.join(' ')

#### 11. Reset index

In [81]:
df.reset_index(drop=True)

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing,Hyundai,2007,80000,45000,Petrol
1,Mahindra Jeep CL550,Mahindra,2006,425000,40,Diesel
2,Hyundai Grand i10,Hyundai,2014,325000,28000,Petrol
3,Ford EcoSport Titanium,Ford,2014,575000,36000,Diesel
4,Ford Figo,Ford,2012,175000,41000,Diesel
...,...,...,...,...,...,...
811,Maruti Suzuki Ritz,Maruti,2011,270000,50000,Petrol
812,Tata Indica V2,Tata,2009,110000,30000,Diesel
813,Toyota Corolla Altis,Toyota,2009,300000,132000,Petrol
814,Tata Zest XM,Tata,2018,260000,27000,Diesel


#### 12. Final data inspection

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 816 entries, 0 to 889
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        816 non-null    object
 1   company     816 non-null    object
 2   year        816 non-null    int32 
 3   Price       816 non-null    int32 
 4   kms_driven  816 non-null    int32 
 5   fuel_type   816 non-null    object
dtypes: int32(3), object(3)
memory usage: 35.1+ KB


In [83]:
df.describe()

,year,Price,kms_driven
count,816.000000,8.160000e+02,816.000000
mean,2012.444853,4.117176e+05,46275.531863
std,4.002992,4.751844e+05,34297.428044
min,1995.000000,3.000000e+04,0.000000
25%,2010.000000,1.750000e+05,27000.000000
50%,2013.000000,2.999990e+05,41000.000000
75%,2015.000000,4.912500e+05,56818.500000
max,2019.000000,8.500003e+06,400000.000000


#### 13.Remove outliers

In [90]:
df = df[df['Price'] < 6e6].reset_index(drop=True)

In [91]:
df.head()

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing,Hyundai,2007,80000,45000,Petrol
1,Mahindra Jeep CL550,Mahindra,2006,425000,40,Diesel
2,Hyundai Grand i10,Hyundai,2014,325000,28000,Petrol
3,Ford EcoSport Titanium,Ford,2014,575000,36000,Diesel
4,Ford Figo,Ford,2012,175000,41000,Diesel


In [92]:
df.shape

(815, 6)

#### 14. Save cleaned data

In [93]:
df.to_csv('cleaned_Car.csv')

## Model Building

#### 15. Prepare features (X) and target (y)

In [95]:
X = df.drop(columns = 'Price')
y = df['Price']

#### 16. Split the data:

In [98]:

X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.2, random_state=42)

## One-Hot Encoding

#### 17. Create and fit the OneHotEncoder:

In [104]:
ohe = OneHotEncoder()
ohe.fit(X[['name','company','fuel_type']])

OneHotEncoder()

In [112]:
ohe.categories_

[array(['Audi A3 Cabriolet', 'Audi A4 1.8', 'Audi A4 2.0', 'Audi A6 2.0',
        'Audi A8', 'Audi Q3 2.0', 'Audi Q5 2.0', 'Audi Q7', 'BMW 3 Series',
        'BMW 5 Series', 'BMW 7 Series', 'BMW X1', 'BMW X1 sDrive20d',
        'BMW X1 xDrive20d', 'Chevrolet Beat', 'Chevrolet Beat Diesel',
        'Chevrolet Beat LS', 'Chevrolet Beat LT', 'Chevrolet Beat PS',
        'Chevrolet Cruze LTZ', 'Chevrolet Enjoy', 'Chevrolet Enjoy 1.4',
        'Chevrolet Sail 1.2', 'Chevrolet Sail UVA', 'Chevrolet Spark',
        'Chevrolet Spark 1.0', 'Chevrolet Spark LS', 'Chevrolet Spark LT',
        'Chevrolet Tavera LS', 'Chevrolet Tavera Neo', 'Datsun GO T',
        'Datsun Go Plus', 'Datsun Redi GO', 'Fiat Linea Emotion',
        'Fiat Petra ELX', 'Fiat Punto Emotion', 'Force Motors Force',
        'Force Motors One', 'Ford EcoSport', 'Ford EcoSport Ambiente',
        'Ford EcoSport Titanium', 'Ford EcoSport Trend',
        'Ford Endeavor 4x4', 'Ford Fiesta', 'Ford Fiesta SXi', 'Ford Figo',
        '

#### 18. Set up column transformer

In [113]:
col_trans = make_column_transformer((OneHotEncoder(categories=ohe.categories_),['name','company','fuel_type']),
            remainder='passthrough')

## Train the Model

#### 19. Initialize and train the model:

In [114]:
lr = LinearRegression()

In [115]:
pipe = make_pipeline(col_trans,lr)

In [116]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Audi A3 Cabriolet', 'Audi A4 1.8', 'Audi A4 2.0', 'Audi A6 2.0',
       'Audi A8', 'Audi Q3 2.0', 'Audi Q5 2.0', 'Audi Q7', 'BMW 3 Series',
       'BMW 5 Series', 'BMW 7 Series', 'BMW X1', 'BMW X1 sDrive20d',
       'BMW X1 xDrive20d', 'Chevrolet Beat', 'Chevrolet Beat...
                                                                            array(['Audi', 'BMW', 'Chevrolet', 'Datsun', 'Fiat', 'Force', 'Ford',
       'Hindustan', 'Honda', 'Hyundai', 'Jaguar', 'Jeep', 'Land',
       'Mahindra', 'Maruti', 'Mercedes', 'Mini', 'Mitsubishi', 'Nissan',
       'Renault', 'Skoda', 'Tata', 'Toyota', 'Volkswagen', 'Volvo'],
      dtype=object),
                                                                            array(['Diesel', 'LPG', 'Petrol'], dtype=object)]),
                                                  ['name', 'company',
                                                   'fuel_type'])])),
                ('linearregression', LinearRegression())])

#### 20. Make predictions and evaluate

In [117]:
y_pred = pipe.predict(X_test)

In [120]:

r2_score(y_test,y_pred)

0.573163946242421

#### 21. Hyperparameter tuning (optional)

In [125]:
s = []
for i in range(1000):    
    X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.2, random_state=i)
    lr = LinearRegression()
    pipe = make_pipeline(col_trans,lr)
    pipe.fit(X_train,y_train)
    y_pred = pipe.predict(X_test)
    s.append(r2_score(y_test,y_pred))

In [127]:
np.argmax(s)

433

In [128]:
s[np.argmax(s)]

0.8457042416667213

In [129]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.2, random_state=np.argmax(s))
lr = LinearRegression()
pipe = make_pipeline(col_trans,lr)
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
r2_score(y_test,y_pred)

0.8457042416667213

## Save the Model

#### 22. Save the trained model

In [131]:
pickle.dump(pipe,open('LinearRegressionModel_CarPrice','wb'))

## Make Predictions with the Model

#### 23.Predict with new data:

In [132]:
pipe.predict(pd.DataFrame([['Maruti Suzuki Swift','Maruti',2019,100,'Petrol']], columns = ['name','company','year','kms_driven','fuel_type']))

array([458900.81022717])